In [58]:
!pip install openpyxl

In [59]:
import psycopg2 as pg2
import pandas as pd
from sqlalchemy import create_engine
import os
from config import api_key

In [62]:
data_file = "Resources/Wines.xlsx"
df = pd.read_excel(data_file, engine='openpyxl')
df.head()

,Vintage,Country,County,Designation,Points,Price,Province,Title,Variety,Winery
0,1919-01-01 00:00:00,Spain,Cava,1919 Brut Selecció,88,$13.00,Catalonia,L'Arboc NV 1919 Brut Selecció Sparkling (Cava),Sparkling Blend,L'Arboc
1,1929-01-01 00:00:00,Italy,Vernaccia di San Gimignano,NaN,87,$14.00,Tuscany,Guidi 1929 2015 Vernaccia di San Gimignano,Vernaccia,Guidi 1929
2,1929-01-01 00:00:00,Italy,Sangiovese di Romagna Superiore,Prugneto,84,$15.00,Central Italy,Poderi dal Nespoli 1929 2011 Prugneto (Sangiov...,Sangiovese,Poderi dal Nespoli 1929
3,1934-01-01 00:00:00,Portugal,NaN,Reserva Velho,93,$495.00,Colares,Adega Viuva Gomes 1934 Reserva Velho Red (Cola...,Ramisco,Adega Viuva Gomes
4,1945-01-01 00:00:00,France,Rivesaltes,Legend Vintage,95,$350.00,Languedoc-Roussillon,Gérard Bertrand 1945 Legend Vintage Red (Rives...,Red Blend,Gérard Bertrand


In [63]:
new_df = df[['Country', 'Points', "Price", 'Province', 'Title', 'Variety', 'Winery']].copy()
new_df.head()

,Country,Points,Price,Province,Title,Variety,Winery
0,Spain,88,$13.00,Catalonia,L'Arboc NV 1919 Brut Selecció Sparkling (Cava),Sparkling Blend,L'Arboc
1,Italy,87,$14.00,Tuscany,Guidi 1929 2015 Vernaccia di San Gimignano,Vernaccia,Guidi 1929
2,Italy,84,$15.00,Central Italy,Poderi dal Nespoli 1929 2011 Prugneto (Sangiov...,Sangiovese,Poderi dal Nespoli 1929
3,Portugal,93,$495.00,Colares,Adega Viuva Gomes 1934 Reserva Velho Red (Cola...,Ramisco,Adega Viuva Gomes
4,France,95,$350.00,Languedoc-Roussillon,Gérard Bertrand 1945 Legend Vintage Red (Rives...,Red Blend,Gérard Bertrand


In [64]:
new_df.count()

Country     24989
Points      24997
Price       23375
Province    24989
Title       24997
Variety     24997
Winery      24997
dtype: int64

In [65]:
new_df = new_df.dropna(how='any')
new_df.count()

Country     23367
Points      23367
Price       23367
Province    23367
Title       23367
Variety     23367
Winery      23367
dtype: int64

In [66]:
new_df.dtypes

Country     object
Points       int64
Price       object
Province    object
Title       object
Variety     object
Winery      object
dtype: object

In [67]:
new_df['Price'] = [float(x.replace("$","").replace(",","")) for x in new_df['Price']]
new_df.dtypes

Country      object
Points        int64
Price       float64
Province     object
Title        object
Variety      object
Winery       object
dtype: object

In [68]:
winexl_df = new_df.loc[(new_df['Variety'] == "Cabernet Sauvignon") | (new_df['Variety'] == "Chardonnay") 
                       | (new_df['Variety'] == "Pinot Gris") | (new_df['Variety'] == "Pinot Noir")
                       | (new_df['Variety'] == "Sauvignon Blanc") | (new_df['Variety'] == "Merlot")]
count = winexl_df['Variety'].value_counts()
count

Pinot Noir            2552
Chardonnay            2167
Cabernet Sauvignon    1834
Sauvignon Blanc        986
Merlot                 587
Pinot Gris             274
Name: Variety, dtype: int64

In [69]:
winexl_df.head()

,Country,Points,Price,Province,Title,Variety,Winery
11,US,89,170.0,California,Sebastiani 1987 Cherryblock Cabernet Sauvignon...,Cabernet Sauvignon,Sebastiani
23,US,82,13.0,California,Gan Eden 1994 Chardonnay (Sonoma County),Chardonnay,Gan Eden
33,South Africa,87,17.0,Stellenbosch,Middelvlei 1995 Cabernet Sauvignon (Stellenbosch),Cabernet Sauvignon,Middelvlei
36,US,83,22.0,California,Meridian 1996 Coastal Reserve Cabernet Sauvign...,Cabernet Sauvignon,Meridian
37,US,84,29.0,Washington,Covey Run 1996 Whiskey Canyon Vyd Cabernet Sau...,Cabernet Sauvignon,Covey Run


In [70]:
winexl_df = winexl_df.reset_index(drop=True)
winexl_df.head()

,Country,Points,Price,Province,Title,Variety,Winery
0,US,89,170.0,California,Sebastiani 1987 Cherryblock Cabernet Sauvignon...,Cabernet Sauvignon,Sebastiani
1,US,82,13.0,California,Gan Eden 1994 Chardonnay (Sonoma County),Chardonnay,Gan Eden
2,South Africa,87,17.0,Stellenbosch,Middelvlei 1995 Cabernet Sauvignon (Stellenbosch),Cabernet Sauvignon,Middelvlei
3,US,83,22.0,California,Meridian 1996 Coastal Reserve Cabernet Sauvign...,Cabernet Sauvignon,Meridian
4,US,84,29.0,Washington,Covey Run 1996 Whiskey Canyon Vyd Cabernet Sau...,Cabernet Sauvignon,Covey Run


In [71]:
csv_file = "Resources/winemag-data_first150k_for_project.csv"
winemag_data_df = pd.read_csv(csv_file)
winemag_data_df.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,France,"A big, powerful wine that sums up the richness...",NaN,99,2300.0,Bordeaux,Pauillac,NaN,Bordeaux-style Red Blend,Château Latour
1,US,The nose on this single-vineyard wine from a s...,Roger Rose Vineyard,91,2013.0,California,Arroyo Seco,Central Coast,Chardonnay,Blair
2,France,"A massive wine for Margaux, packed with tannin...",NaN,98,1900.0,Bordeaux,Margaux,NaN,Bordeaux-style Red Blend,Château Margaux
3,France,A wine that has created its own universe. It h...,Clos du Mesnil,100,1400.0,Champagne,Champagne,NaN,Chardonnay,Krug
4,France,A wine that has created its own universe. It h...,Clos du Mesnil,100,1400.0,Champagne,Champagne,NaN,Chardonnay,Krug


In [72]:
new_winemag_data_df = winemag_data_df[['country', 'description', 'points', 'price', 'province', 'variety', 'winery']].copy()
new_winemag_data_df.head()

,country,description,points,price,province,variety,winery
0,France,"A big, powerful wine that sums up the richness...",99,2300.0,Bordeaux,Bordeaux-style Red Blend,Château Latour
1,US,The nose on this single-vineyard wine from a s...,91,2013.0,California,Chardonnay,Blair
2,France,"A massive wine for Margaux, packed with tannin...",98,1900.0,Bordeaux,Bordeaux-style Red Blend,Château Margaux
3,France,A wine that has created its own universe. It h...,100,1400.0,Champagne,Chardonnay,Krug
4,France,A wine that has created its own universe. It h...,100,1400.0,Champagne,Chardonnay,Krug


In [73]:
# Check for missing information
new_winemag_data_df.count()

country        150925
description    150930
points         150930
price          137235
province       150925
variety        150930
winery         150930
dtype: int64

In [74]:
# Drop all rows with missing information
new_winemag_data_df = new_winemag_data_df.dropna(how='any')

In [75]:
# Check that all columns have the same amount of rows now. 
new_winemag_data_df.count()

country        137230
description    137230
points         137230
price          137230
province       137230
variety        137230
winery         137230
dtype: int64

In [76]:
# Check that the data types fit the data. 
new_winemag_data_df.dtypes

country         object
description     object
points           int64
price          float64
province        object
variety         object
winery          object
dtype: object

In [77]:
# look at the values in the 'variety' column.
new_winemag_data_df['variety'].value_counts()

Chardonnay               13775
Pinot Noir               13625
Cabernet Sauvignon       12671
Red Blend                 9377
Sauvignon Blanc           6054
                         ...  
Baga-Touriga Nacional        1
Chardonel                    1
Sarba                        1
Erbaluce                     1
Rufete                       1
Name: variety, Length: 619, dtype: int64

In [78]:
new_winemag_data_df['variety'] = new_winemag_data_df['variety'].replace(
    {'Pinot Grigio': 'Pinot Gris'})

In [79]:
new_winemag_df = new_winemag_data_df.loc[(new_winemag_data_df['variety'] == "Cabernet Sauvignon") | (new_winemag_data_df['variety'] == "Chardonnay") 
                       | (new_winemag_data_df['variety'] == "Pinot Gris") | (new_winemag_data_df['variety'] == "Pinot Noir")
                       | (new_winemag_data_df['variety'] == "Sauvignon Blanc") | (new_winemag_data_df['variety'] == "Merlot")]
count = new_winemag_df['variety'].value_counts()
count

Chardonnay            13775
Pinot Noir            13625
Cabernet Sauvignon    12671
Sauvignon Blanc        6054
Merlot                 4987
Pinot Gris             2545
Name: variety, dtype: int64

In [80]:
new_winemag_df.head()

,country,description,points,price,province,variety,winery
1,US,The nose on this single-vineyard wine from a s...,91,2013.0,California,Chardonnay,Blair
3,France,A wine that has created its own universe. It h...,100,1400.0,Champagne,Chardonnay,Krug
4,France,A wine that has created its own universe. It h...,100,1400.0,Champagne,Chardonnay,Krug
5,France,A wine that has created its own universe. It h...,100,1400.0,Champagne,Chardonnay,Krug
26,France,From arguably the finest white wine vineyard i...,98,757.0,Burgundy,Chardonnay,Bouchard Père & Fils


In [81]:
new_winemag_df = new_winemag_df.reset_index(drop=True)
new_winemag_df.head()

,country,description,points,price,province,variety,winery
0,US,The nose on this single-vineyard wine from a s...,91,2013.0,California,Chardonnay,Blair
1,France,A wine that has created its own universe. It h...,100,1400.0,Champagne,Chardonnay,Krug
2,France,A wine that has created its own universe. It h...,100,1400.0,Champagne,Chardonnay,Krug
3,France,A wine that has created its own universe. It h...,100,1400.0,Champagne,Chardonnay,Krug
4,France,From arguably the finest white wine vineyard i...,98,757.0,Burgundy,Chardonnay,Bouchard Père & Fils


In [35]:
#Create connection with PostgreSQL
conn = pg2.connect(database='Wines',user='postgres', password=api_key)

#Establish connection and start cursor to be ready to query
cur = conn.cursor()

In [36]:
#Pass in a query as a string
cur.execute("SELECT * FROM wine_info")

#Show results of first row
cur.fetchone()

('1/1/1919',
 'Spain',
 'Cava',
 '1919 Brut Selecció',
 88,
 '$13.00 ',
 'Catalonia',
 "L'Arboc NV 1919 Brut Selecció Sparkling (Cava)",
 'Sparkling Blend',
 "L'Arboc")

In [37]:
#Pulling the wanted data
Salaries = pd.read_sql("SELECT * FROM wine_info", conn)

#Show results
Salaries.head()

,vintage,country,county,designation,points,price,province,title,variety,winery
0,1/1/1919,Spain,Cava,1919 Brut Selecció,88,$13.00,Catalonia,L'Arboc NV 1919 Brut Selecció Sparkling (Cava),Sparkling Blend,L'Arboc
1,1/1/1929,Italy,Vernaccia di San Gimignano,None,87,$14.00,Tuscany,Guidi 1929 2015 Vernaccia di San Gimignano,Vernaccia,Guidi 1929
2,1/1/1929,Italy,Sangiovese di Romagna Superiore,Prugneto,84,$15.00,Central Italy,Poderi dal Nespoli 1929 2011 Prugneto (Sangiov...,Sangiovese,Poderi dal Nespoli 1929
3,1/1/1934,Portugal,None,Reserva Velho,93,$495.00,Colares,Adega Viuva Gomes 1934 Reserva Velho Red (Cola...,Ramisco,Adega Viuva Gomes
4,1/1/1945,France,Rivesaltes,Legend Vintage,95,$350.00,Languedoc-Roussillon,Gérard Bertrand 1945 Legend Vintage Red (Rives...,Red Blend,Gérard Bertrand


In [32]:
#Pass in a query as a string
cur.execute("SELECT * FROM wine_reviews")

#Show results of first row
cur.fetchone()

('Gallo',
 '13312834',
 'Ecco Domani174 Pinot Grigio - 750ml Bottle',
 '2017-08-01T21:13:49.000Z',
 '5',
 'This a fantastic white wine for any occasion!',
 'My Favorite White Wine',
 'Bjh')

In [33]:
#Pulling the wanted data
Salaries = pd.read_sql("SELECT * FROM wine_reviews", conn)

#Show results
Salaries.head()

,brand,manufacturernumber,winename,reviewsdate,reviewsrating,reviewstext,reviewstitle,reviewsusername
0,Gallo,13312834,Ecco Domani174 Pinot Grigio - 750ml Bottle,2017-08-01T21:13:49.000Z,5,This a fantastic white wine for any occasion!,My Favorite White Wine,Bjh
1,Fresh Craft Co.,50392800,Fresh Craft174 Mango Citrus - 4pk / 250ml Bottle,2017-07-10T04:17:34.000Z,5,"Tart, not sweet...very refreshing and delicious!",Yum!!,Wino
2,1000 Stories,50399893,1000 Stories174 Zinfandel - 750ml Bottle,2017-10-02T12:52:27.000Z,5,I was given this wine so it was a delightful s...,A New Favorite!,Bama Mom
3,1000 Stories,50399893,1000 Stories174 Zinfandel - 750ml Bottle,2016-11-02T10:00:37.000Z,5,This is a phenomenal wine and my new favorite ...,"Bold, Flavorful, Aromatic, Delicious",Av Dub
4,Wine Cube153,14831433,Pink Moscato - 3l Bottle - Wine Cube153,2014-11-25T03:37:34.000Z,5,4 750ml bottles for the price of two With way ...,"Yum! Plus, Environmentally Friendly!",Chelseamay


In [24]:
#Pass in a query as a string
cur.execute("SELECT * FROM wines_by_state_seven")

#Show results of first row
cur.fetchone()

('ARIZONA', '65,413', '0', '0')

In [25]:
#Pulling the wanted data
Salaries = pd.read_sql("SELECT * FROM wines_by_state_seven", conn)

#Show results
Salaries.head()

,usstate,wines,specialwines,specialwinestaxable
0,ARIZONA,"65,413",0,0
1,CALIFORNIA,"568,801,873",794158,171939
2,COLORADO,"221,258",0,0
3,CONNECTICUT,"110,484",0,0
4,FLORIDA,"1,610,275",0,0


In [26]:
#Pass in a query as a string
cur.execute("SELECT * FROM wines_by_state_fifteen")

#Show results of first row
cur.fetchone()

('ALABAMA', '42,656', '0', '0')

In [27]:
#Pulling the wanted data
Salaries = pd.read_sql("SELECT * FROM wines_by_state_fifteen", conn)

#Show results
Salaries.head()

,usstate,wines,specialwines,specialwinestaxable
0,ALABAMA,"42,656",0,0
1,ARIZONA,"198,746","5,050","2,133"
2,ARKANSAS,"203,701",0,0
3,CALIFORNIA,"638,173,762","844,056","158,470"
4,COLORADO,"449,679",0,0


In [53]:
rds_connection_string = "postgres:Planning4future@@localhost:5432/Wines"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [54]:
#from sqlalchemy.dialects import postgresql
#print(insert_stmt.compile(dialect=postgresql.dialect()))

In [82]:
 engine.table_names()

['newwinemag',
 'winexl',
 'wine_info',
 'wines_by_state_seven',
 'wines_by_state_fifteen',
 'winemag',
 'wine_reviews']

In [83]:
 ['winexl', 'newwinemag']

['winexl', 'newwinemag']

###  Connect to local database

In [90]:
 winexl_df.to_sql(name='winexl', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "Country" of relation "winexl" does not exist
LINE 1: INSERT INTO winexl ("Country", "Points", "Price", "Province"...
                            ^

[SQL: INSERT INTO winexl ("Country", "Points", "Price", "Province", "Title", "Variety", "Winery") VALUES (%(Country)s, %(Points)s, %(Price)s, %(Province)s, %(Title)s, %(Variety)s, %(Winery)s)]
[parameters: ({'Country': 'US', 'Points': 89, 'Price': 170.0, 'Province': 'California', 'Title': 'Sebastiani 1987 Cherryblock Cabernet Sauvignon (Sonoma Valley)', 'Variety': 'Cabernet Sauvignon', 'Winery': 'Sebastiani'}, {'Country': 'US', 'Points': 82, 'Price': 13.0, 'Province': 'California', 'Title': 'Gan Eden 1994 Chardonnay (Sonoma County)', 'Variety': 'Chardonnay', 'Winery': 'Gan Eden'}, {'Country': 'South Africa', 'Points': 87, 'Price': 17.0, 'Province': 'Stellenbosch', 'Title': 'Middelvlei 1995 Cabernet Sauvignon (Stellenbosch)', 'Variety': 'Cabernet Sauvignon', 'Winery': 'Middelvlei'},

###  Check for tables

In [87]:
new_winemag_df.to_sql(name='newwinemag', con=engine, if_exists='append', index=False)

### Use pandas to load csv converted DataFrame into database

In [88]:
pd.read_sql_query('select * from winexl', con=engine).head()

,country,points,price,province,title,variety,winery


### Confirm data has been added by querying the wines table

In [89]:
pd.read_sql_query('select * from newwinemag', con=engine).head()

,country,description,points,price,province,variety,winery
0,US,The nose on this single-vineyard wine from a s...,91,2013.0,California,Chardonnay,Blair
1,France,A wine that has created its own universe. It h...,100,1400.0,Champagne,Chardonnay,Krug
2,France,A wine that has created its own universe. It h...,100,1400.0,Champagne,Chardonnay,Krug
3,France,A wine that has created its own universe. It h...,100,1400.0,Champagne,Chardonnay,Krug
4,France,From arguably the finest white wine vineyard i...,98,757.0,Burgundy,Chardonnay,Bouchard Père & Fils
